In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.cm import rainbow
import seaborn as sns
from scipy import stats
from scipy.stats import skew
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('application_data.csv')

## EDA

In [3]:
# Etapa 01 - Tratamento e limpeza dos dados nulos

docs_colunas = ['FLAG_DOCUMENT_' + str(i) for i in range(2, 22)]
df['NUM_DOCS_APRESENTADOS'] = df[docs_colunas].sum(axis=1)
flag_docs_colunas = [col for col in df.columns if col.startswith('FLAG_DOCUMENT_')]
df_exclusao_cols = df.drop(columns=flag_docs_colunas)
df = df_exclusao_cols

# Exclusão de colunas com valores nulos

def excluir_colunas_nulas(df, percentual):
    dados_faltantes = df.isnull().mean() * 100
    col_para_apagar = dados_faltantes[dados_faltantes > percentual].index.tolist()
    return col_para_apagar

percentual = 0
col_para_apagar = excluir_colunas_nulas(df, percentual)

# Exclui as colunas identificadas
df_exclusao_colunas = df.drop(columns=col_para_apagar)
df = df_exclusao_colunas

# Exclusão de colunas de categoria única

columns_to_drop = ['FLAG_MOBIL',
                   'FLAG_EMP_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL','REG_REGION_NOT_WORK_REGION',
                   'REG_REGION_NOT_LIVE_REGION','REG_CITY_NOT_WORK_CITY','REG_CITY_NOT_LIVE_CITY']
df = df.drop(columns_to_drop,axis=1)

# Exclusão dos valores 'XNA' (essencialmente nulos) em CODE_GENDER

df = df[df['CODE_GENDER'] != 'XNA']

# Conversao de algumas colunas para anos

# Colunas que terão o sinal removido (valores absolutos)
colunas_para_remover_sinal = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']
df[colunas_para_remover_sinal] = df[colunas_para_remover_sinal].abs()

# Engenharia de atributos
df['IDADE'] = df['DAYS_BIRTH'] // 365
df['ANOS_TRABALHADOS'] = df['DAYS_EMPLOYED'] // 365
df['ANOS_REGISTRO'] = df['DAYS_REGISTRATION'] // 365
df['ANOS_PUBLICACAO_ID'] = df['DAYS_ID_PUBLISH'] // 365

# Remover colunas originais
df = df.drop(colunas_para_remover_sinal, axis=1)

#  Devido a uma alta correlação, decidi remover Region_Rating_Client_With_City

column_to_drop = ['REGION_RATING_CLIENT_W_CITY']
df1 = df.drop(column_to_drop,axis=1)

## Random Forest 1 - Base após EDA

In [7]:
# Selecionar colunas numéricas (int64 e float64)
colunas_numericas = df1.select_dtypes(include=['int64', 'float64']).columns

df_arvore_1 = df1[colunas_numericas]
print('base para árvore: ', df_arvore_1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 0 to 307510
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307507 non-null  int64  
 1   TARGET                       307507 non-null  int64  
 2   CNT_CHILDREN                 307507 non-null  int64  
 3   AMT_INCOME_TOTAL             307507 non-null  float64
 4   AMT_CREDIT                   307507 non-null  float64
 5   REGION_POPULATION_RELATIVE   307507 non-null  float64
 6   FLAG_WORK_PHONE              307507 non-null  int64  
 7   FLAG_PHONE                   307507 non-null  int64  
 8   REGION_RATING_CLIENT         307507 non-null  int64  
 9   HOUR_APPR_PROCESS_START      307507 non-null  int64  
 10  LIVE_REGION_NOT_WORK_REGION  307507 non-null  int64  
 11  LIVE_CITY_NOT_WORK_CITY      307507 non-null  int64  
 12  NUM_DOCS_APRESENTADOS        307507 non-null  int64  
 13  IDAD

In [8]:
# Separando os dados entre variáveis explicativas (X) e a variável alvo (y)
X = df_arvore_1.drop(columns=['TARGET'])  # Substitua 'TARGET' pela coluna alvo correta
y = df_arvore_1['TARGET']

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Criando o modelo de Random Forest
random_forest = RandomForestClassifier(random_state=42)

# Treinando o modelo com os dados de treino
random_forest.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = random_forest.predict(X_test)

# Avaliação dos resultados
print(f'Acurácia: {accuracy_score(y_test, y_pred):.4f}')
print(f'Precisão: {precision_score(y_test, y_pred):.4f}')
print(f'Recall: {recall_score(y_test, y_pred):.4f}')
print(f'F1-Score: {f1_score(y_test, y_pred):.4f}')
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.9192
Precisão: 0.0000
Recall: 0.0000
F1-Score: 0.0000
Matriz de Confusão:
[[84803     2]
 [ 7448     0]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     84805
           1       0.00      0.00      0.00      7448

    accuracy                           0.92     92253
   macro avg       0.46      0.50      0.48     92253
weighted avg       0.85      0.92      0.88     92253



In [ ]:
# tentativa de melhoria

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

# Etapa 02 - Criação do modelo Random Forest com melhorias
# Separação das features e target
X = df_arvore_1.drop(columns=['TARGET'])  # Supondo que 'TARGET' seja a variável-alvo
y = df_arvore_1['TARGET']

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Criando o modelo de Random Forest com balanceamento de classes
random_forest = RandomForestClassifier(random_state=42, class_weight='balanced')

# Parâmetros a serem ajustados no GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Aplicando o GridSearch com validação cruzada
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1, verbose=1)

# Treinando o modelo
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

# Fazendo previsões no conjunto de teste
y_pred = best_model.predict(X_test)

# Avaliação dos resultados
print(f"Melhores parâmetros: {grid_search.best_params_}")
print(f'Acurácia: {accuracy_score(y_test, y_pred):.4f}')
print(f'Precisão: {precision_score(y_test, y_pred):.4f}')
print(f'Recall: {recall_score(y_test, y_pred):.4f}')
print(f'F1-Score: {f1_score(y_test, y_pred):.4f}')
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 108 candidates, totalling 540 fits


## Árvore 2 - Base completa

In [13]:
# Carregar os dados
df2 = pd.read_csv('application_data.csv')

In [14]:
variables_to_test = df2.select_dtypes(include=['int64', 'float64']).columns

alpha = 0.05
significant_variables = []

for variable in variables_to_test:
    # Extrair dados para os grupos de inadimplência (TARGET = 1) e não inadimplência (TARGET = 0)
    data_default = df2[df2['TARGET'] == 1][variable].dropna()  # Remover valores nulos
    data_no_default = df2[df2['TARGET'] == 0][variable].dropna()  # Remover valores nulos

    # Realizar o teste t para duas amostras
    t_statistic, p_value = stats.ttest_ind(data_default, data_no_default, equal_var=False)

    # Verificar significância
    if p_value < alpha:
        significant_variables.append(variable)

# Criar um novo DataFrame apenas com as variáveis significativas
df2 = df2[significant_variables]

def identify_columns_to_drop(df, percentage):
    missing_percentage = df.isnull().mean() * 100
    columns_to_drop = missing_percentage[missing_percentage > percentage].index.tolist()
    return columns_to_drop

percentage = 0
columns_to_drop = identify_columns_to_drop(df2, percentage)

# Exclui as colunas identificadas
df2 = df2.drop(columns=columns_to_drop)

In [16]:
# Separando os dados entre variáveis explicativas (X) e a variável alvo (y)
X = df2.drop(columns=['TARGET'])  # Substitua 'TARGET' pela coluna alvo correta
y = df2['TARGET']

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Criando o modelo de Random Forest
random_forest = RandomForestClassifier(random_state=42)

# Treinando o modelo com os dados de treino
random_forest.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = random_forest.predict(X_test)

# Avaliação dos resultados
print(f'Acurácia: {accuracy_score(y_test, y_pred):.4f}')
print(f'Precisão: {precision_score(y_test, y_pred):.4f}')
print(f'Recall: {recall_score(y_test, y_pred):.4f}')
print(f'F1-Score: {f1_score(y_test, y_pred):.4f}')
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.9193
Precisão: 0.0000
Recall: 0.0000
F1-Score: 0.0000
Matriz de Confusão:
[[84805     1]
 [ 7448     0]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     84806
           1       0.00      0.00      0.00      7448

    accuracy                           0.92     92254
   macro avg       0.46      0.50      0.48     92254
weighted avg       0.85      0.92      0.88     92254

